In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'NanumGothic'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
major_s = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장주력_stand.csv', index_col=0)
major_test = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240318상장주력test_stand.csv', index_col=0)

In [20]:
major = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317_상장주력전처리완료(거시합침).csv', index_col=0)
minor = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317_상장비주력전처리완료(거시합침).csv', index_col=0)

In [4]:
major_s.shape

(4757, 153)

In [ ]:
# major_s.drop(['회사명',
#   '회계년도',
#   '설립일',
#   '상장일',
#   '상장폐지일',
#   '상장협 산업분류(소분류)_x',
#   '상장협 산업분류(중분류)_x',
#   '회계년',
#   'Unnamed: 0'], axis=1, inplace=True)

In [ ]:
# major_test.drop(['회사명',
#   '회계년도',
#   '설립일',
#   '상장일',
#   '상장폐지일',
#   '상장협 산업분류(소분류)_x',
#   '상장협 산업분류(중분류)_x',
#   '회계년',
#   'Unnamed: 0'], axis=1, inplace=True)

##### train_df, test_df 설정

In [7]:
train_df = major_s[['CASH FLOW 대 매출액비율',
                    '매출액순이익률',
                    '1주당매출액',
                    '자기자본순이익률',
                    '적립금비율(재정비율)',
                    '자본',
                    '재고자산 대 유동자산비율',
                    '매출채권회전기간',
                    '비유동부채비율',
                    '순운전자본회전률',
                    '유보율',
                    'CASH FLOW 대 차입금비율',
                    '유동부채비율',
                    '부실']]

In [13]:
test_df = major_test[['CASH FLOW 대 매출액비율',
                    '매출액순이익률',
                    '1주당매출액',
                    '자기자본순이익률',
                    '적립금비율(재정비율)',
                    '자본',
                    '재고자산 대 유동자산비율',
                    '매출채권회전기간',
                    '비유동부채비율',
                    '순운전자본회전률',
                    '유보율',
                    'CASH FLOW 대 차입금비율',
                    '유동부채비율',
                    '부실']]

In [14]:
train_df.reset_index(drop=True,inplace=True)

In [15]:
x_train = train_df.drop("부실",axis=1)
y_train = train_df["부실"]
x_test =test_df.drop("부실",axis=1)
y_test = test_df["부실"]

In [16]:
from imblearn.under_sampling import RandomUnderSampler

# RandomUnderSampler 객체 생성
under = RandomUnderSampler(random_state=4, sampling_strategy=0.1)

# RandomUnderSampler를 사용하여 샘플링된 특징 및 타겟 데이터 생성
x_resampled, y_resampled = under.fit_resample(x_train, y_train)

y_resampled.value_counts()

0.0    880
1.0     88
Name: 부실, dtype: int64

In [ ]:
# from imblearn.over_sampling import RandomOverSampler

# x_resampled2, y_resampled2 = RandomOverSampler(random_state=4, sampling_strategy=1).fit_resample(x_resampled, y_resampled )
# print(x_train.shape)
# y_resampled2.value_counts()

(3485, 13)


0.0    540
1.0    540
Name: 부실, dtype: int64

In [17]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    pr_score = average_precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

##### 교차검증

In [5]:
def label_split(df):
  # feature = df.drop(fix_list, axis=1)
  feature = df
  label = df['부실']
  return feature, label

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate(y_true, y_pred):
    # 정확도 (Accuracy)
    accuracy = accuracy_score(y_true, y_pred)
    print("Accuracy:", accuracy)

    # 정밀도 (Precision)
    precision = precision_score(y_true, y_pred)
    print("Precision:", precision)

    # 재현율 (Recall)
    recall = recall_score(y_true, y_pred)
    print("Recall:", recall)

    # F1 스코어 (F1 Score)
    f1 = f1_score(y_true, y_pred)
    print("F1 Score:", f1)

    # AUC-ROC
    try:
        auc_roc = roc_auc_score(y_true, y_pred)
        print("AUC-ROC:", auc_roc)
    except ValueError:
        print("AUC-ROC cannot be calculated for multiclass classification.")

In [20]:
!pip install catboost

In [21]:
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


def modeling_cv(train, test,threshold=0.5):
    print(train['부실'].value_counts())

    feature, label = label_split(train)
    test_feature, test_label = label_split(test)

    feature, label = label_split(train)
    test_feature, test_label = label_split(test)

    print(label.value_counts())
    print(test_label.value_counts())

    models = [
        # ("Logistic Regression", LogisticRegression()),
        # ("KNN", KNeighborsClassifier(n_neighbors=5)),
        # ("Random Forest", RandomForestClassifier(n_estimators=100, random_state=42)),
        # ("XGBoost", xgb.XGBClassifier()),
        # ("LightGBM", lgb.LGBMClassifier()),
        ("CatBoost", CatBoostClassifier()),
        ("SVM (Linear Kernel)", SVC(kernel='linear', probability=True)),
    ]

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # Adjust n_splits as needed

    print("Stratified K-Fold Cross-Validation Performance:")
    for name, model in models:
        print(f"\n---{name}---")
        for train_index, val_index in cv.split(feature, label):
            X_train, X_val = feature.iloc[train_index], feature.iloc[val_index]
            y_train, y_val = label.iloc[train_index], label.iloc[val_index]

            undersampler = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
            X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

            print(y_train_resampled.value_counts())

            model.fit(X_train_resampled, y_train_resampled)  # Train the model on the training fold

            # Use trained model to make predictions on the validation fold
            if hasattr(model, "predict_proba"):
                y_pred_prob = model.predict_proba(X_val)[:, 1]
                y_pred = (y_pred_prob >= threshold).astype(int)  # Apply threshold
            else:
                y_pred = model.predict(X_val)
            print(f"---{name}---")
            evaluate(y_val, y_pred)
            print()

In [22]:
modeling_cv(train_df, test_df,threshold=0.3 )

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
103:	learn: 0.0318827	total: 433ms	remaining: 3.73s
104:	learn: 0.0308563	total: 434ms	remaining: 3.7s
105:	learn: 0.0303842	total: 438ms	remaining: 3.69s
106:	learn: 0.0296507	total: 440ms	remaining: 3.67s
107:	learn: 0.0292503	total: 444ms	remaining: 3.67s
108:	learn: 0.0286849	total: 448ms	remaining: 3.66s
109:	learn: 0.0281258	total: 452ms	remaining: 3.66s
110:	learn: 0.0275387	total: 456ms	remaining: 3.65s
111:	learn: 0.0266834	total: 458ms	remaining: 3.63s
112:	learn: 0.0263180	total: 462ms	remaining: 3.62s
113:	learn: 0.0258675	total: 465ms	remaining: 3.62s
114:	learn: 0.0253384	total: 469ms	remaining: 3.61s
115:	learn: 0.0249646	total: 473ms	remaining: 3.6s
116:	learn: 0.0244203	total: 476ms	remaining: 3.59s
117:	learn: 0.0239753	total: 478ms	remaining: 3.57s
118:	learn: 0.0233882	total: 480ms	remaining: 3.55s
119:	learn: 0.0229218	total: 483ms	remaining: 3.54s
120:	learn: 0.0223612	total: 485ms	remaining: 3.52s
121:	learn: 0.0220245	total: 4

In [31]:
def calculate_f1_score(recall, precision):
    """
    Calculate F1-score using recall and precision.

    Parameters:
    - recall: Recall score.
    - precision: Precision score.

    Returns:
    - f1: F1-score.
    """
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [32]:
# F1-score 계산
f1 = calculate_f1_score(1.0, 1.0)
print("F1-score:", f1)

F1-score: 1.0


##### 1. RandomForest

In [26]:
# 랜덤포레스트 구현
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=52)

rf_clf.fit(x_resampled, y_resampled)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)

In [27]:
print("랜덤포레스트")
get_clf_eval(y_test, rf_pred)

랜덤포레스트
오차행렬
[[1730    1]
 [  19    0]]
정확도: 0.9886, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.4997, PR score : 0.0109


##### 하이퍼파라미터 조정 & 임계값 설정한 뒤 모델링

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# RandomForestClassifier 모델 생성
rf_clf = RandomForestClassifier()

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝과 교차 검증 수행
grid_search = GridSearchCV(rf_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_rf_clf = grid_search.best_estimator_

# 테스트 데이터에 대한 예측 수행
rf_pred = best_rf_clf.predict(x_test)

In [28]:
# 임계값 설정
threshold = 0.25
rf_pred = (rf_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("랜덤포레스트")
get_clf_eval(y_test, rf_pred)

랜덤포레스트
오차행렬
[[1721   10]
 [  17    2]]
정확도: 0.9846, 정밀도 : 0.1667, 재현율:0.1053,F1 스코어:0.1290
ROC 스코어: 0.5497, PR score : 0.0273


In [29]:
# 최적의 모델과 하이퍼파라미터 가져오기
best_rf_clf = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}


##### 2. LogisticRegression

In [30]:
# 로지스틱 회귀 모델
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(x_resampled, y_resampled)
lr_pred = lr_clf.predict(x_test)
lr_pred_proba = lr_clf.predict_proba(x_test)

In [31]:
print("로지스틱")
get_clf_eval(y_test, lr_pred)

로지스틱
오차행렬
[[1722    9]
 [  17    2]]
정확도: 0.9851, 정밀도 : 0.1818, 재현율:0.1053,F1 스코어:0.1333
ROC 스코어: 0.5500, PR score : 0.0289


In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# 그리드서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'C': [0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 1.0, 10.0],
    'penalty': ['l1', 'l2']
}

# 로지스틱 회귀 모델 생성
logistic_clf = LogisticRegression()

# 그리드서치 수행
grid_search = GridSearchCV(logistic_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_logistic_clf = grid_search.best_estimator_

# 예측 확률 계산
logistic_pred_proba = best_logistic_clf.predict_proba(x_test)

In [33]:
# 임계값 설정 (예시로 0.5 사용)
threshold = 0.1
logistic_pred = (logistic_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("로지스틱")
get_clf_eval(y_test, logistic_pred)

로지스틱
오차행렬
[[1640   91]
 [   9   10]]
정확도: 0.9429, 정밀도 : 0.0990, 재현율:0.5263,F1 스코어:0.1667
ROC 스코어: 0.7369, PR score : 0.0573


In [34]:
# 최적의 모델과 하이퍼파라미터 가져오기
best_logistic_clf = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'C': 0.3, 'penalty': 'l2'}


##### 3. SVM

In [35]:
# SVM 모형 구현
from sklearn import svm

svm_clf = svm.SVC(kernel='linear', probability=True)  # 선형 SVM 모델 생성
svm_clf.fit(x_resampled, y_resampled)  # 훈련 데이터를 이용하여 모델 훈련
# 테스트 데이터를 사용하여 예측
svm_pred = svm_clf.predict(x_test)
svm_pred_proba = svm_clf.predict_proba(x_test)

In [36]:
print("SVM")
get_clf_eval(y_test, svm_pred)

SVM
오차행렬
[[1721   10]
 [  17    2]]
정확도: 0.9846, 정밀도 : 0.1667, 재현율:0.1053,F1 스코어:0.1290
ROC 스코어: 0.5497, PR score : 0.0273


In [67]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# RandomizedSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}
svm_clf = SVC(probability=True)
random_search = RandomizedSearchCV(svm_clf, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_svm_clf = random_search.best_estimator_

# 예측 확률 계산
svm_pred_proba = best_svm_clf.predict_proba(x_test)

In [38]:
# 임계값 설정 (예시로 0.5 사용)
threshold = 0.3
svm_pred = (svm_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("SVM")
get_clf_eval(y_test, svm_pred)

SVM
오차행렬
[[1726    5]
 [  19    0]]
정확도: 0.9863, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.4986, PR score : 0.0109


In [68]:
# 최적의 모델과 하이퍼파라미터 가져오기
best_svm_clf = random_search.best_estimator_
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'kernel': 'poly', 'gamma': 'auto', 'C': 0.1}


##### 4. KNN

In [61]:
# KNN 모형 구현
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(x_resampled, y_resampled)
knn_pred = knn_clf.predict(x_test)
knn_pred_proba = knn_clf.predict_proba(x_test)

In [41]:
print("knn")
get_clf_eval(y_test, knn_pred)

knn
오차행렬
[[1718   13]
 [  17    2]]
정확도: 0.9829, 정밀도 : 0.1333, 재현율:0.1053,F1 스코어:0.1176
ROC 스코어: 0.5489, PR score : 0.0237


In [65]:
# RandomizedSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]  # 1은 맨해튼 거리, 2는 유클리디언 거리
}
knn_clf = KNeighborsClassifier()
random_search = RandomizedSearchCV(knn_clf, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_knn_clf = random_search.best_estimator_

# 예측 확률 계산
knn_pred_proba = best_knn_clf.predict_proba(x_test)

In [63]:
# 임계값 설정
threshold = 0.2
knn_pred = (knn_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("KNN")
get_clf_eval(y_test, knn_pred)

KNN
오차행렬
[[1656   75]
 [  12    7]]
정확도: 0.9503, 정밀도 : 0.0854, 재현율:0.3684,F1 스코어:0.1386
ROC 스코어: 0.6625, PR score : 0.0383


In [66]:
# 최적의 모델과 하이퍼파라미터 가져오기
best_knn_clf = random_search.best_estimator_
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'weights': 'uniform', 'p': 2, 'n_neighbors': 7, 'algorithm': 'ball_tree'}


##### 5. LGBM

In [45]:
# LGBM 모형 구현
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier()
lgb_model.fit(x_resampled, y_resampled)
# 테스트 데이터를 사용하여 예측
lgb_pred = lgb_model.predict(x_test)
lgb_pred_proba = lgb_model.predict_proba(x_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 88, number of negative: 880
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000375 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3212
[LightGBM] [Info] Number of data points in the train set: 968, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.090909 -> initscore=-2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [46]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

# 그리드서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5]
}

# LGBMClassifier 모델 생성
lgbm_clf = LGBMClassifier()

# 그리드서치 수행
grid_search = GridSearchCV(lgbm_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_lgbm_clf = grid_search.best_estimator_

# 예측 확률 계산
lgbm_pred = best_lgbm_clf.predict(x_test)
lgbm_pred_proba = best_lgbm_clf.predict_proba(x_test)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [47]:
# 임계값 설정
threshold = 0.2
lgb_pred = (lgb_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("LGBM")
get_clf_eval(y_test, lgb_pred)

LGBM
오차행렬
[[1729    2]
 [  19    0]]
정확도: 0.9880, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.4994, PR score : 0.0109


In [48]:
# 최적의 모델과 하이퍼파라미터 가져오기
best_lgbm_clf = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 200}


##### 6. xgboost

In [49]:
# xgboost 모형 구현
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance

xgb_wrapper = XGBClassifier()

xgb_wrapper.fit(x_resampled, y_resampled, verbose = True)


xgb_pred = xgb_wrapper.predict(x_test)
xgb_pred_proba = xgb_wrapper.predict_proba(x_test)

In [50]:
print("xgb")
get_clf_eval(y_test, xgb_pred)

xgb
오차행렬
[[1731    0]
 [  19    0]]
정확도: 0.9891, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0109


In [51]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# 그리드서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5]
}

# XGBClassifier 모델 생성
xgb_clf = XGBClassifier()

# 그리드서치 수행
grid_search = GridSearchCV(xgb_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_xgb_clf = grid_search.best_estimator_

# 테스트 데이터에 대한 예측 수행
xgb_pred = best_xgb_clf.predict(x_test)
xgb_pred_proba = best_xgb_clf.predict_proba(x_test)

In [52]:
# 임계값 설정
threshold = 0.2
xgb_pred = (xgb_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("xgboost")
get_clf_eval(y_test, lgb_pred)

xgboost
오차행렬
[[1729    2]
 [  19    0]]
정확도: 0.9880, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.4994, PR score : 0.0109


In [53]:
# 최적의 모델과 하이퍼파라미터 가져오기
best_xgb_clf = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 300}


#### 7. catboost

In [54]:
!pip install catboost

In [55]:
from catboost import CatBoostClassifier

catboost_model = CatBoostClassifier()
catboost_model.fit(x_resampled, y_resampled)
# 테스트 데이터를 사용하여 예측
catboost_pred = catboost_model.predict(x_test)
catboost_pred_proba = catboost_model.predict_proba(x_test)

Learning rate set to 0.01016
0:	learn: 0.6746549	total: 6.83ms	remaining: 6.83s
1:	learn: 0.6609853	total: 11.3ms	remaining: 5.64s
2:	learn: 0.6435868	total: 15.5ms	remaining: 5.16s
3:	learn: 0.6263877	total: 23ms	remaining: 5.73s
4:	learn: 0.6107680	total: 28ms	remaining: 5.58s
5:	learn: 0.5944359	total: 32.2ms	remaining: 5.34s
6:	learn: 0.5791782	total: 36.2ms	remaining: 5.14s
7:	learn: 0.5646114	total: 40.2ms	remaining: 4.99s
8:	learn: 0.5505727	total: 44.2ms	remaining: 4.86s
9:	learn: 0.5377824	total: 48.1ms	remaining: 4.76s
10:	learn: 0.5238880	total: 52.2ms	remaining: 4.7s
11:	learn: 0.5105162	total: 56.5ms	remaining: 4.65s
12:	learn: 0.4985487	total: 60.5ms	remaining: 4.6s
13:	learn: 0.4863518	total: 64.9ms	remaining: 4.57s
14:	learn: 0.4763145	total: 69ms	remaining: 4.53s
15:	learn: 0.4665534	total: 73.2ms	remaining: 4.5s
16:	learn: 0.4557503	total: 77.7ms	remaining: 4.49s
17:	learn: 0.4475600	total: 81.8ms	remaining: 4.46s
18:	learn: 0.4389228	total: 86ms	remaining: 4.44s
19:	

In [56]:
print("catboost")
get_clf_eval(y_test, catboost_pred)

catboost
오차행렬
[[1730    1]
 [  19    0]]
정확도: 0.9886, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.4997, PR score : 0.0109


In [57]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

# 그리드서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'depth': [3, 5, 7],
    'l2_leaf_reg': [1, 3, 5]
}

# CatBoostClassifier 모델 생성
catboost_clf = CatBoostClassifier()

# 그리드서치 수행
grid_search = GridSearchCV(catboost_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_catboost_clf = grid_search.best_estimator_

# 테스트 데이터에 대한 예측 수행
catboost_pred = best_catboost_clf.predict(x_test)
catboost_pred_proba = best_catboost_clf.predict_proba(x_test)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0:	learn: 0.6755857	total: 6.33ms	remaining: 1.89s
1:	learn: 0.6631988	total: 12.9ms	remaining: 1.93s
2:	learn: 0.6468685	total: 19.6ms	remaining: 1.94s
3:	learn: 0.6318216	total: 26.7ms	remaining: 1.97s
4:	learn: 0.6167076	total: 33.4ms	remaining: 1.97s
5:	learn: 0.6053365	total: 40ms	remaining: 1.96s
6:	learn: 0.5921583	total: 46.9ms	remaining: 1.96s
7:	learn: 0.5795759	total: 54.1ms	remaining: 1.97s
8:	learn: 0.5664426	total: 61.1ms	remaining: 1.98s
9:	learn: 0.5514378	total: 68.1ms	remaining: 1.98s
10:	learn: 0.5410840	total: 74.7ms	remaining: 1.96s
11:	learn: 0.5300254	total: 81.6ms	remaining: 1.96s
12:	learn: 0.5195500	total: 88.3ms	remaining: 1.95s
13:	learn: 0.5082176	total: 94.9ms	remaining: 1.94s
14:	learn: 0.4980561	total: 101ms	remaining: 1.93s
15:	learn: 0.4885909	total: 108ms	remaining: 1.93s
16:	learn: 0.4792645	total: 120ms	remaining: 2s
17:	learn: 0.4662772	total: 131ms	remaining: 2.05s
18:	learn: 0.4566017	total: 137ms	remaining: 2.

In [58]:
# 임계값 설정
threshold = 0.2
catboost_pred = (catboost_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("catboost")
get_clf_eval(y_test, catboost_pred)

catboost
오차행렬
[[1724    7]
 [  17    2]]
정확도: 0.9863, 정밀도 : 0.2222, 재현율:0.1053,F1 스코어:0.1429
ROC 스코어: 0.5506, PR score : 0.0331


In [59]:
# 최적의 모델과 하이퍼파라미터 가져오기
best_catboost_clf = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'depth': 3, 'iterations': 200, 'l2_leaf_reg': 3, 'learning_rate': 0.01}


# 모델링

In [ ]:
# # 트리기반 모델링을 위한 스케일링 전 데이터
# train_scaling_x = pd.read_csv('/content/drive/MyDrive/공유문서함/비상장외감기업/주력_train윈저1퍼.csv', index_col=0)
# train_scaling_x.drop('설립일', axis=1, inplace=True)

# test_scaling_x = pd.read_csv('/content/drive/MyDrive/공유문서함/비상장외감기업/주력_test.csv', index_col=0)

# train_no_scaling_m =train_scaling_x[final_list_m]
# train_no_scaling_s =train_scaling_x[final_list_s]

# test_no_scaling_m =test_scaling_x[final_list_m]
# test_no_scaling_s =test_scaling_x[final_list_s]

In [ ]:
# fix_list = ['회사명','거래소코드' ,'회계년도','부실', '감사의견코드','상장협 산업분류(중분류)','상장협 산업분류(소분류)']

# def train_test_split(df):
#   feature = df.drop(fix_list, axis=1)
#   label = df['부실']
#   return feature, label

### 성능지표

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate(y_true, y_pred):
    # 정확도 (Accuracy)
    accuracy = accuracy_score(y_true, y_pred)
    print("Accuracy:", accuracy)

    # 정밀도 (Precision)
    precision = precision_score(y_true, y_pred)
    print("Precision:", precision)

    # 재현율 (Recall)
    recall = recall_score(y_true, y_pred)
    print("Recall:", recall)

    # F1 스코어 (F1 Score)
    f1 = f1_score(y_true, y_pred)
    print("F1 Score:", f1)

    # AUC-ROC
    try:
        auc_roc = roc_auc_score(y_true, y_pred)
        print("AUC-ROC:", auc_roc)
    except ValueError:
        print("AUC-ROC cannot be calculated for multiclass classification.")

## 1. LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression(random_state=52)
lr = lr.fit(x_train, y_train)

pred = lr.predict(x_test)

In [ ]:
# 랜덤포레스트 구현
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=52)

rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def logistic(train, test):
  feature, label = train_test_split(train)
  test_feature, test_label = train_test_split(test)

  lr = LogisticRegression()
  lr = lr.fit(feature, label)

  pred = lr.predict(test_feature)

  return test_label, pred


In [ ]:
# minmax 데이터
test, pred = logistic(minmax_selected_feature, test_m)
evaluate(y_test, y_pred)

Accuracy: 0.9782176760615203
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC-ROC: 0.4999623890476907


In [ ]:
test_m['부실'].value_counts()

0    13294
1      295
Name: 부실, dtype: int64

In [ ]:
# standard 데이터
test,pred = logistic(standard_selected_feature, test_s)
evaluate(test, pred)

Accuracy: 0.9786592096548679
Precision: 0.5714285714285714
Recall: 0.06779661016949153
F1 Score: 0.12121212121212119
AUC-ROC: 0.5333341408001061


## 거리기반 SVM, KNN

#### SVM

In [ ]:
from sklearn import svm

def svm_model(train, test):
  feature, label = train_test_split(train)
  test_feature, test_label = train_test_split(test)

  clf = svm.SVC(kernel='linear')  # 선형 SVM 모델 생성
  clf.fit(feature, label)  # 훈련 데이터를 이용하여 모델 훈련

  # 테스트 데이터를 사용하여 예측
  y_pred = clf.predict(test_feature)

  return test_label, y_pred

In [ ]:
# minmax
test, pred = svm_model(minmax_selected_feature, test_m)
evaluate(test, pred)

Accuracy: 0.978291264993745
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC-ROC: 0.5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# standard
test, pred = svm_model(standard_selected_feature, test_s)
evaluate(test, pred)

Accuracy: 0.978291264993745
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC-ROC: 0.5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

def knn_model(train, test):
  feature, label = train_test_split(train)
  test_feature, test_label = train_test_split(test)

  # KNN 모델 생성 및 훈련
  k = 5  # K값 설정
  clf = KNeighborsClassifier(n_neighbors=k)
  clf.fit(feature, label)

  # 테스트 데이터를 사용하여 예측
  y_pred = clf.predict(test_feature)

  # 정확도 출력
  return test_label, y_pred

In [ ]:
test, pred = knn_model(minmax_selected_feature, test_m)
evaluate(test, pred)

Accuracy: 0.9781440871292958
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC-ROC: 0.4999247780953814


In [ ]:
test, pred = knn_model(standard_selected_feature, test_s)
evaluate(test, pred)

Accuracy: 0.9778497314003973
Precision: 0.40625
Recall: 0.04406779661016949
F1 Score: 0.07951070336391437
AUC-ROC: 0.5213192902112078


## 트리기반 model

#### RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def randomForest(train, test):
  feature, label = train_test_split(train)
  test_feature, test_label = train_test_split(test)

  rf = RandomForestClassifier(n_estimators=100, random_state=42)
  rf.fit(feature, label)

  y_pred = rf.predict(test_feature)

  return test_label, y_pred


In [ ]:
test, pred = randomForest(minmax_selected_feature, test_m)
evaluate(test, pred)

Accuracy: 0.978291264993745
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC-ROC: 0.5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test, pred = randomForest(standard_selected_feature, test_s)
evaluate(test, pred)

Accuracy: 0.978291264993745
Precision: 0.5
Recall: 0.04406779661016949
F1 Score: 0.0809968847352025
AUC-ROC: 0.5215449559250637


In [ ]:
test, pred = randomForest(train_no_scaling_m, test_no_scaling_m)
evaluate(test, pred)

Accuracy: 0.9792479211126647
Precision: 0.5414012738853503
Recall: 0.288135593220339
F1 Score: 0.3761061946902655
AUC-ROC: 0.641359808043899


In [ ]:
test, pred = randomForest(train_no_scaling_s, test_no_scaling_s)
evaluate(test, pred)

Accuracy: 0.9804253440282581
Precision: 0.5900621118012422
Recall: 0.3220338983050847
F1 Score: 0.4166666666666667
AUC-ROC: 0.6585346263001278


#### XGBoost

In [ ]:
import xgboost as xgb

def xgboost(train, test):
  feature, label = train_test_split(train)
  test_feature, test_label = train_test_split(test)

  xgb_model = xgb.XGBClassifier()
  xgb_model.fit(feature, label)

  # 테스트 데이터를 사용하여 예측
  y_pred = xgb_model.predict(test_feature)

  return test_label, y_pred

In [ ]:
test, pred = xgboost(minmax_selected_feature, test_m)
evaluate(test, pred)

Accuracy: 0.9715946721613069
Precision: 0.23699421965317918
Recall: 0.13898305084745763
F1 Score: 0.17521367521367523
AUC-ROC: 0.5645268797188996


In [ ]:
test, pred = xgboost(standard_selected_feature, test_s)
evaluate(test, pred)

Accuracy: 0.9784384428581941
Precision: 0.5217391304347826
Recall: 0.08135593220338982
F1 Score: 0.14076246334310852
AUC-ROC: 0.53985052515089


In [ ]:
test, pred = xgboost(train_no_scaling_m, test_no_scaling_m)
evaluate(test, pred)

Accuracy: 0.9771874310103761
Precision: 0.45901639344262296
Recall: 0.2847457627118644
F1 Score: 0.3514644351464435
AUC-ROC: 0.6386493970773103


In [ ]:
test, pred = xgboost(train_no_scaling_s, test_no_scaling_s)
evaluate(test, pred)

Accuracy: 0.9783648539259695
Precision: 0.5026737967914439
Recall: 0.31864406779661014
F1 Score: 0.3900414937759336
AUC-ROC: 0.655824215333539


#### LGBM

In [ ]:
import lightgbm as lgb

def lightgbm(train, test):

  feature, label = train_test_split(train)
  test_feature, test_label = train_test_split(test)

  lgb_model = lgb.LGBMClassifier()
  lgb_model.fit(feature, label)

  # 테스트 데이터를 사용하여 예측
  y_pred = lgb_model.predict(test_feature)
  return test_label, y_pred


In [ ]:
test, pred = lightgbm(minmax_selected_feature, test_m)
evaluate(test, pred)

[LightGBM] [Info] Number of positive: 859, number of negative: 36331
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003286 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 37190, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.023098 -> initscore=-3.744658
[LightGBM] [Info] Start training from score -3.744658
Accuracy: 0.978291264993745
Precision: 0.5
Recall: 0.003389830508474576
F1 Score: 0.006734006734006735
AUC-ROC: 0.501657304301928


In [ ]:
test, pred = lightgbm(standard_selected_feature, test_s)
evaluate(test, pred)

[LightGBM] [Info] Number of positive: 859, number of negative: 36331
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002872 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 37190, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.023098 -> initscore=-3.744658
[LightGBM] [Info] Start training from score -3.744658
Accuracy: 0.9788799764515417
Precision: 0.6111111111111112
Recall: 0.07457627118644068
F1 Score: 0.13293051359516617
AUC-ROC: 0.5367615822608899


In [ ]:
test, pred = lightgbm(train_no_scaling_m, test_no_scaling_m)
evaluate(test, pred)

[LightGBM] [Info] Number of positive: 859, number of negative: 36331
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003550 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 37190, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.023098 -> initscore=-3.744658
[LightGBM] [Info] Start training from score -3.744658
Accuracy: 0.9785120317904187
Precision: 0.5083798882681564
Recall: 0.30847457627118646
F1 Score: 0.3839662447257384
AUC-ROC: 0.6509275243323737


In [ ]:
test, pred = lightgbm(train_no_scaling_s, test_no_scaling_s)
evaluate(test, pred)

[LightGBM] [Info] Number of positive: 859, number of negative: 36331
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002817 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 37190, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.023098 -> initscore=-3.744658
[LightGBM] [Info] Start training from score -3.744658
Accuracy: 0.9778497314003973
Precision: 0.4835164835164835
Recall: 0.2983050847457627
F1 Score: 0.3689727463312369
AUC-ROC: 0.6456171128558059


#### CatBoost

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 2.9 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

def catboost(train, test):

  feature, label = train_test_split(train)
  test_feature, test_label = train_test_split(test)

  catboost_model = CatBoostClassifier()
  catboost_model.fit(feature, label)

  # 테스트 데이터를 사용하여 예측
  y_pred = catboost_model.predict(test_feature)
  return test_label, y_pred


In [ ]:
test, pred = catboost(minmax_selected_feature, test_m)
evaluate(test, pred)

Learning rate set to 0.048251
0:	learn: 0.5694458	total: 22ms	remaining: 21.9s
1:	learn: 0.4722124	total: 34.1ms	remaining: 17s
2:	learn: 0.4056114	total: 44.9ms	remaining: 14.9s
3:	learn: 0.3422984	total: 59.3ms	remaining: 14.8s
4:	learn: 0.2876995	total: 70.6ms	remaining: 14.1s
5:	learn: 0.2465071	total: 81.7ms	remaining: 13.5s
6:	learn: 0.2118099	total: 96ms	remaining: 13.6s
7:	learn: 0.1850797	total: 107ms	remaining: 13.3s
8:	learn: 0.1681951	total: 120ms	remaining: 13.2s
9:	learn: 0.1486038	total: 131ms	remaining: 13s
10:	learn: 0.1341030	total: 142ms	remaining: 12.8s
11:	learn: 0.1227827	total: 153ms	remaining: 12.6s
12:	learn: 0.1128223	total: 165ms	remaining: 12.5s
13:	learn: 0.1046713	total: 176ms	remaining: 12.4s
14:	learn: 0.0981626	total: 189ms	remaining: 12.4s
15:	learn: 0.0929385	total: 202ms	remaining: 12.4s
16:	learn: 0.0878825	total: 214ms	remaining: 12.4s
17:	learn: 0.0837038	total: 226ms	remaining: 12.3s
18:	learn: 0.0804358	total: 237ms	remaining: 12.3s
19:	learn: 0

In [ ]:
test, pred = catboost(standard_selected_feature, test_s)
evaluate(test, pred)

Learning rate set to 0.048251
0:	learn: 0.5756332	total: 17.6ms	remaining: 17.6s
1:	learn: 0.4884855	total: 31.5ms	remaining: 15.7s
2:	learn: 0.4059968	total: 74.5ms	remaining: 24.8s
3:	learn: 0.3386556	total: 128ms	remaining: 31.8s
4:	learn: 0.2912889	total: 160ms	remaining: 31.9s
5:	learn: 0.2496790	total: 209ms	remaining: 34.6s
6:	learn: 0.2148214	total: 226ms	remaining: 32.1s
7:	learn: 0.1858864	total: 247ms	remaining: 30.6s
8:	learn: 0.1631156	total: 262ms	remaining: 28.8s
9:	learn: 0.1451554	total: 282ms	remaining: 27.9s
10:	learn: 0.1305613	total: 317ms	remaining: 28.5s
11:	learn: 0.1192209	total: 345ms	remaining: 28.4s
12:	learn: 0.1093248	total: 403ms	remaining: 30.6s
13:	learn: 0.1012226	total: 472ms	remaining: 33.2s
14:	learn: 0.0946818	total: 542ms	remaining: 35.6s
15:	learn: 0.0893932	total: 634ms	remaining: 39s
16:	learn: 0.0848108	total: 718ms	remaining: 41.5s
17:	learn: 0.0809022	total: 782ms	remaining: 42.7s
18:	learn: 0.0777096	total: 837ms	remaining: 43.2s
19:	learn:

In [ ]:
test, pred = catboost(train_no_scaling_m, test_no_scaling_m)
evaluate(test, pred)

Learning rate set to 0.048251
0:	learn: 0.5694458	total: 51.1ms	remaining: 51s
1:	learn: 0.4722124	total: 110ms	remaining: 54.7s
2:	learn: 0.4056114	total: 134ms	remaining: 44.5s
3:	learn: 0.3422984	total: 215ms	remaining: 53.5s
4:	learn: 0.2876995	total: 283ms	remaining: 56.3s
5:	learn: 0.2465071	total: 337ms	remaining: 55.8s
6:	learn: 0.2118099	total: 390ms	remaining: 55.3s
7:	learn: 0.1850797	total: 446ms	remaining: 55.3s
8:	learn: 0.1681951	total: 486ms	remaining: 53.5s
9:	learn: 0.1486038	total: 521ms	remaining: 51.6s
10:	learn: 0.1341030	total: 561ms	remaining: 50.4s
11:	learn: 0.1227827	total: 594ms	remaining: 48.9s
12:	learn: 0.1128223	total: 641ms	remaining: 48.7s
13:	learn: 0.1046713	total: 687ms	remaining: 48.4s
14:	learn: 0.0981626	total: 715ms	remaining: 46.9s
15:	learn: 0.0929385	total: 750ms	remaining: 46.1s
16:	learn: 0.0878825	total: 783ms	remaining: 45.3s
17:	learn: 0.0837038	total: 825ms	remaining: 45s
18:	learn: 0.0804358	total: 867ms	remaining: 44.8s
19:	learn: 0.0

In [ ]:
test, pred = catboost(train_no_scaling_s, test_no_scaling_s)
evaluate(test, pred)

Learning rate set to 0.048251
0:	learn: 0.5756332	total: 11.3ms	remaining: 11.3s
1:	learn: 0.4884855	total: 24.8ms	remaining: 12.4s
2:	learn: 0.4059968	total: 37.2ms	remaining: 12.4s
3:	learn: 0.3386556	total: 48ms	remaining: 12s
4:	learn: 0.2912889	total: 59.2ms	remaining: 11.8s
5:	learn: 0.2496790	total: 70.8ms	remaining: 11.7s
6:	learn: 0.2148214	total: 82.1ms	remaining: 11.6s
7:	learn: 0.1858864	total: 104ms	remaining: 12.9s
8:	learn: 0.1631156	total: 126ms	remaining: 13.9s
9:	learn: 0.1451554	total: 139ms	remaining: 13.7s
10:	learn: 0.1305613	total: 152ms	remaining: 13.7s
11:	learn: 0.1192209	total: 164ms	remaining: 13.5s
12:	learn: 0.1093248	total: 175ms	remaining: 13.3s
13:	learn: 0.1012226	total: 186ms	remaining: 13.1s
14:	learn: 0.0946818	total: 202ms	remaining: 13.2s
15:	learn: 0.0893932	total: 213ms	remaining: 13.1s
16:	learn: 0.0848108	total: 224ms	remaining: 12.9s
17:	learn: 0.0809022	total: 235ms	remaining: 12.8s
18:	learn: 0.0777096	total: 246ms	remaining: 12.7s
19:	lear